In [2]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import time
import datetime as dt
import pandas as pd
import numpy as np
import json
import requests
import csv
import os
import glob

In [40]:
'''
Fields
market_cap_usd
name
price_usd
symbol
'''
exclusion_list = ['BCC','XRP']
capital = 20000

response = requests.get('https://api.coinmarketcap.com/v1/ticker/')
json_data = json.loads(response.text)
i = 0
crypto_df = {}
for cc in json_data:
    temp_df = pd.DataFrame(cc, index = [i])[['symbol','name','market_cap_usd','price_usd']]
    if len(crypto_df) == 0:
        crypto_df = temp_df
    else:
        crypto_df = pd.concat([crypto_df,temp_df],axis = 0)
    i += 1
    
crypto_df['market_cap_usd'] = pd.to_numeric(crypto_df['market_cap_usd'])
crypto_df['price_usd'] = pd.to_numeric(crypto_df['price_usd'])

crypto_df = crypto_df.sort_values(by = 'market_cap_usd', ascending = False)
crypto_df = crypto_df[crypto_df['symbol'].isin(exclusion_list) == False][:30]

crypto_df['weight'] = crypto_df['market_cap_usd']/crypto_df['market_cap_usd'].sum()
crypto_df['dollar_weight'] = crypto_df['weight']*capital
crypto_df['expected_holdings'] = crypto_df['dollar_weight']/crypto_df['price_usd']

In [41]:
crypto_df

,symbol,name,market_cap_usd,price_usd,weight,dollar_weight,expected_holdings
0,BTC,Bitcoin,1.122062e+11,6729.230000,0.625792,12515.845469,1.859922
1,ETH,Ethereum,2.897235e+10,302.906000,0.161584,3231.670023,10.668887
2,BCH,Bitcoin Cash,1.566346e+10,933.483000,0.087358,1747.153230,1.871650
4,LTC,Litecoin,3.251275e+09,60.468300,0.018133,362.657800,5.997486
5,DASH,Dash,2.576705e+09,335.471000,0.014371,287.414079,0.856748
6,NEO,NEO,1.861600e+09,28.640000,0.010382,207.648918,7.250311
7,XEM,NEM,1.807704e+09,0.200856,0.010082,201.637183,1003.889268
8,XMR,Monero,1.670487e+09,108.926000,0.009317,186.331519,1.710625
9,ETC,Ethereum Classic,1.435958e+09,14.755200,0.008009,160.171396,10.855251
10,MIOTA,IOTA,1.415467e+09,0.509247,0.007894,157.885843,310.037847
